# Hello

Welcome to the SME Pavement Crack Detection Interactive Notebook. Please run the following code to import all of the functions.

In [ ]:
import numpy as np  # May not need in the future.
import os
from skimage import io
from skimage.filters import threshold_multiotsu
from src.CreatePaths import createPaths
from src.LoadImage import loadImage
from src.SaveCutImages import saveCutImages
from src.ThresholdCutImages import thresholdCutImages
from src.SaveThresholdedImages import saveThresholdedImages
# We do not need argparser since this is interactive.
# from src.ArgParser import parseArgs
from src.TestImages import loadLabeledImages, saveLabeledCutImages, testImages

# Globals

The following are definitions of global paramters. Change them as need be, but best to leave them alone.

In [ ]:
# Feel free to change
DIVIDE_X = 25  # Number of divisions on one of axies
DIVIDE_Y = 25  # Number of diisions on other axis

# Try not to change
dir_path = os.path.realpath('.')  # The directory of the project.

# Do not change
thresholding_function = threshold_multiotsu  # Function for thresholding.


# Creating Paths

The following code is to create the different sub folders that will be required to run the rest of the code.

In [ ]:
IMAGES = os.listdir(os.path.join(dir_path , "data" , "raw_images"))

required_paths = [  # The folders that must exist.
    "\\data",
    "\\data\\raw_images",  # Where the .tif files will be stored.
    "\\results",
]
createPaths(required_paths, dir_path)

In [ ]:
image_dict = {}
for image in IMAGES:
    # Name the image
    image_dict[image] = {}
    image_dict[image]["file_name"] = image  # Redundent for the sake of clarity
    image_dict[image]["name"] = image.split('.')[0]  # Without the .tif
    
    # Name the paths
    image_dict[image]["raw_path"] = os.path.join("data" , "raw_images", image_dict[image]["file_name"])
    image_dict[image]["cut_path"] = os.path.join("results" , image_dict[image]["name"], "raw_cut_images")
    image_dict[image]["thresholded_cut_path"] = os.path.join("results" ,  image_dict[image]["name"] , "thresholded_cut_images")
    image_dict[image]["labeled_cut_path"] = os.path.join("results" ,  image_dict[image]["name"] , "labeled_cut_images")

In [ ]:
for key in image_dict:
    required_paths.append(image_dict[image]["cut_path"])
    required_paths.append(image_dict[image]["thresholded_cut_path"])
    required_paths.append(image_dict[image]["labeled_cut_path"])
createPaths(required_paths, dir_path)

# Cut and Threshold Images

Running the cell below will cut and threshold the images insdie of the `data/raw_images` folder.

In [ ]:
for key in image_dict:
    image_dict[key]["cut_image_dict"] = loadImage(image_dict[key]["raw_path"], DIVIDE_X, DIVIDE_Y)
    image_dict[key]["thresholded_dict"] = thresholdCutImages(image_dict[key]["cut_image_dict"], thresholding_function)

After running the cell above, the image has now been cut and thresholded. You may want to run the cell below in order to save the cut images and the thresholded images in their respective paths.

In [ ]:
# Saves the cut images
saveDictionary(image_dict[key]["cut_image_dict"], dir_path, image_dict[key]["cut_path"], "cut_image")

In [ ]:
saveDictionary(image_dict[key]["thresholded_dict"], dir_path, image_dict[key]["thresholded_cut_path"], "cut_image")

# Temporary Functions

These functions are temporarily stored down here. This allows rapid prototyping by keeping functions being worked on inside the same file as the interactive code. These should be ran immediately after the imports.

In [ ]:
def createPaths(paths, dir_path):
    for path in paths:
        if not os.path.exists(os.path.join(dir_path , path)):
            os.makedirs(os.path.join(dir_path , path))

In [ ]:
def thresholdCutImages(cut_img_dict, thresholding_function):
    thresholded_dict = {}
    for key in cut_img_dict:
        if len(np.unique(cut_img_dict[key])) < 3:
            thresholded_dict[key] = None 
        else:                         
            image = np.dot(cut_img_dict[key][...,:3], [0.299, 0.587, 0.114])
            thresholds = thresholding_function(image)
            regions = np.digitize(image, bins=thresholds)
            thresholded_dict[key] = regions == 0
        
        # Would like a more elegant solution in the future.
        if type(thresholded_dict[key]) == type(None):
            thresholded_dict[key] = np.full(cut_img_dict[key].shape, False)
    
    return thresholded_dict

In [ ]:
def saveDictionary(dictionary, dir_path, folder, name):
    """
    Args:
        Dictionary: A dictionary where the keys are tupples of (x,y) and the value is an image as a numpy array.
        Folder: Path to folder where the file will be saved.
        Name: Naming convention for the files.
    Output:
        None
        Will save the images inside Dictionary into Folder with the name = name + str(key[0]*1000 + key[1])[1:] + ".tif"
    """
    for key in dictionary:
        file_name = name + "_" + "{:02d}{:02d}".format(key[0], key[1]) + ".tif"
        io.imsave(os.path.join(dir_path, folder, file_name), dictionary[key], check_contrast=False)

In [ ]:
# Temporary cell for debugging these functions

def loadLabeledImages(path, images, color=(0,0,0,255)):
    """
    Loads in the the hand traced images and creates a binary image 
    of black pixels vs rest
    Args:
        path - the path to the folder containing ONLY the images.
        images - the path to the file containing the images labeled.
        color - the color the images were labeled in.
    """
    text = open(images,'r')
    labeled_images = [line for line in text.readlines() ]

    labeled_image_dict = {}
    for file in os.listdir(path):
        if file[-4:] in labeled_images:
            key = (int(file[-4:-2]), int(file[-2:]))
            img = io.imread(path + '/' + file)
            labeled_image_dict[key] = img == color
    return labeled_image_dict

def saveLabeledCutImages(labeled_image_dict, dir_path):
    for key in labeled_image_dict:
        name = "{:02d}{:02d}".format(key[0], key[1])
        io.imsave(dir_path + "/results/labeled_cut_images/labeled_img_" + name + ".tif", labeled_image_dict[key], check_contrast=False)
    return None

def testImages(labeled_image_dict, thresholded_dict):
    score_dict = {}
    for key in labeled_image_dict:
        lt = labeled_image_dict[key]
        tt = thresholded_dict[key]
        union = lt | tt
        intersection = lt & tt
        ans = intersection / union 
        score_dict[key] = ans
    return score_dict
    